<div>
    <img src="images/emlyon.png" style="height:60px; float:left; padding-right:10px; margin-top:5px" />
    <span>
        <h1 style="padding-bottom:5px;"> Introduction to Deep Learning </h1>
        <a href="https://masters.em-lyon.com/fr/msc-in-data-science-artificial-intelligence-strategy">[DSAIS]</a> MSc in Data Science & Artificial Intelligence Strategy <br/>
         Paris | © Saeed VARASTEH
    </span>
</div>

## Lecture 07 : First RNN Model

In this notebook, we will build our first RNN model and try to understand its mechanics. We will heavily use the content from our third lecture (notebook) in the model training section.

<img style="width:60%" src="./images/rnn_unrolled.png" />

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

np.random.seed(72)
torch.manual_seed(72)
device = 'cpu'

---

### Data Generation

Let’s start generating some synthetic data: we start with a vector of 1000 points from the __sine__ function

In [ ]:
N = 1000
series = np.sin(0.1*np.arange(N)) + np.random.randn(N)*0.1

plt.plot(series)
plt.show()

let's see if we can use __L__ past values to predict the next value.

In [ ]:
L = 10
X = []
y = []
for t in range(len(series) - L):
    x_ = series[t:t+L]
    X.append(x_)
    y_ = series[t+L]
    y.append(y_)

X = np.array(X).reshape(-1, L, 1)
y = np.array(y).reshape(-1, 1)

N = len(X)
print("X.shape", X.shape, "Y.shape", y.shape)

Next, let’s split our synthetic data into train and test sets:

In [ ]:
X_train = torch.from_numpy(X[:-N//2].astype(np.float32))
y_train = torch.from_numpy(y[:-N//2].astype(np.float32))
X_test = torch.from_numpy(X[-N//2:].astype(np.float32))
y_test = torch.from_numpy(y[-N//2:].astype(np.float32))

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

### Building a Model

Let’s build a proper (yet simple) RNN model for this regression task:

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(input_size = 1, hidden_size = 5, num_layers = 1, batch_first=True)
        self.fc = nn.Linear(5, 1)
        
    def forward(self, x):
        
        # initial hidden states
        h0 = torch.zeros(1, x.size(0), 5).to(device) # nb of layers, batch_size, number of hidden units 
        
        out, _ = self.rnn(x, h0)
        
        # out is of size (batch size, sequence length, nb of hidden units)
        # we only want the out at the final time step
        out = out[:, -1, :]
        
        out = self.fc(out)
        return out 

#### Understanding RNN Shapes

In [ ]:
model = SimpleRNN()

inputs = X_train[:16]

outputs = model(inputs)

inputs.shape, outputs.shape

### Model Training

In [ ]:
model = SimpleRNN().to(device)

lr = 1e-1
n_epochs = 200

loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=lr)

train_losses = []

for epoch in range(n_epochs):
    model.train()
    
    x_batch = X_train.to(device) 
    y_batch = y_train.to(device)
    
    yhat = model(x_batch)
    loss = loss_fn(yhat, y_batch)
    
    train_losses.append(loss.item())
                        
    optimizer.zero_grad()
    loss.backward()    
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{n_epochs}, Train Loss: {loss.item():.4f}')

print('Done')

In [ ]:
plt.plot(train_losses,  label="train loss");
plt.legend()

### Making Predications

In [ ]:
y_pred = model( X_test.to(device) )
y_pred = y_pred.detach().numpy()

plt.plot(y_test[:], c="b", label="actual data");
plt.plot(y_pred[:], c="r", label="predicted data");
plt.legend()

---